*von Benedikt Funke*

## Auslesen und Speichern der Daten von Kafka

Um die an Kafka gesendeten Daten zu verarbeiten ist ein Consumer notwendig, der diese entgegennimmt und weiterverarbeitet. Hier wird dafür zunächst eine Subscription auf das Topic `weather` in Kafka mithilfe der `KafkaConsumer`-Klasse angelegt. Wie auch schon im `kafka-producer.ipynb` gilt auch hier, dass bei keinen weiteren Parameter versucht wird eine Verbindung mit Kafka auf `localhost` herzustellen.

Außerdem wird eine Verbindung zu der `weather`-Collection in der MongoDB hergestellt und eventuell vorhandene alte Daten werden gelöscht, so dass vor dem Einfügen eine leere Collection vorhanden ist.

In [ ]:
from kafka import KafkaConsumer
from pymongo import MongoClient
import datetime as dt

consumer = KafkaConsumer('weather')

client = MongoClient('mongodb://localhost:27017/')
weather = client.bda.weather
weather.delete_many({})

Im Folgenden erfolgt die Auswertung der erhaltenen Nachrichten aus Kafka und die Aufbereitung für die Abspeicherung in der MongoDB.
Dafür wird zunächst versucht, die Werte der CSV-Formatierten Zeile zu extrahieren. Anschließend wird die `station_id` in einen `Integer` und die Temperatur in ein `Float` konvertiert. Außerdem wird versucht das Datum aus dem String des 3. Wertes zu extrahieren und als Datumsobjekt abzuspeichern
Falls eine dieser Konvertierungen fehlschlägt wird der Datensatz übersprungen und ein Fehler wird ausgegeben.

Abschließend werden die konvertieren Werte in die `weather`-Collection der MongoDB abgespeichert.

In [ ]:
for message in consumer:
    try:
        values = message.value.decode('utf-8').split(', ')
        station_id = int(values[1].strip("\'"))
        temperature = float(values[5].strip("\'"))
        date = dt.datetime.strptime(values[2].strip("\'"),"%Y-%m-%d %H:%M:%S")
    except:
        print('An Error occurred processing data. Skipping...')
        continue

    weather.insert_one({
        'station_id': station_id,
        'date': date,
        'temperature': temperature
        })